In [1]:
import json
import pandas as pd
import numpy as np
import random

file_path = "yelp_academic_dataset_review.json"

line_count = len(open("yelp_academic_dataset_review.json").readlines())



In [2]:



user_ids,business_ids,stars = [],[],[]


count_for_example = 0

with open('yelp_academic_dataset_review.json','r',encoding = 'utf-8') as json_file:

    for line in json_file:
        
        statement = json.loads(line)
              
        user_ids += [statement["user_id"]]
        business_ids += [statement["business_id"]]
        stars += [statement["stars"]]
        
            



In [3]:



ratings = pd.DataFrame(
{"user_id" : user_ids, "business_id" : business_ids, "rating" : stars}
)




print(ratings)
print(ratings.values)



                        user_id             business_id  rating
0        Xqd0DzHaiyRqVH3WRG7hzg  vcNAWiLM4dR7D2nwwJ7nCA       5
1        H1kH6QZV7Le4zqTRNxoZow  vcNAWiLM4dR7D2nwwJ7nCA       2
2        zvJCcrpm2yOZrxKffwGQLA  vcNAWiLM4dR7D2nwwJ7nCA       4
3        KBLW4wJA_fwoWmMhiHRVOA  vcNAWiLM4dR7D2nwwJ7nCA       4
4        zvJCcrpm2yOZrxKffwGQLA  vcNAWiLM4dR7D2nwwJ7nCA       4
...                         ...                     ...     ...
1125453  lhMo-dGq0V2iKqBIiwUJSg  BVxlrYWgmi-8TPGMe6CTpg       5
1125454  TTrzXCtB2MZA8Azw56bRlw  BVxlrYWgmi-8TPGMe6CTpg       5
1125455  rtS7mDof5d-cEPBsmVuUJw  BVxlrYWgmi-8TPGMe6CTpg       3
1125456  tZs84cKAUSOtP_nAiSdreQ  BVxlrYWgmi-8TPGMe6CTpg       4
1125457  LfnB4N7SVSAIPOM3If_kDA  BVxlrYWgmi-8TPGMe6CTpg       3

[1125458 rows x 3 columns]
[['Xqd0DzHaiyRqVH3WRG7hzg' 'vcNAWiLM4dR7D2nwwJ7nCA' 5]
 ['H1kH6QZV7Le4zqTRNxoZow' 'vcNAWiLM4dR7D2nwwJ7nCA' 2]
 ['zvJCcrpm2yOZrxKffwGQLA' 'vcNAWiLM4dR7D2nwwJ7nCA' 4]
 ...
 ['rtS7mDof5d-cEPBsmVuUJw' 'BVxlrY

In [4]:
ratings_frac = ratings.sample(frac=0.10)

training_set = ratings_frac.sample(frac=0.7)

test_set = ratings_frac.drop(training_set.index)



In [5]:

#ratings 모델을 matrix 형태로 변환

ratings_training = training_set.pivot_table(
   index ='user_id',columns = 'business_id', values = 'rating',aggfunc= "median"   #같은 유저가 같은 business에 대한 평가를 할 때 median으로 사용
).fillna(0)



In [6]:
ratings_test = test_set.pivot_table(
   index ='user_id',columns = 'business_id', values = 'rating',aggfunc= "median"   #같은 유저가 같은 business에 대한 평가를 할 때 median으로 사용
).fillna(0)

In [7]:
ratings = ratings_training.to_numpy()

print(ratings)

print(ratings.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(47340, 21127)


In [11]:
class MatrixFactorization():
    
    def __init__(self, rating,latent, learning_rate, weight, epochs ,verbose = True):
        
        self.rating = rating
        self.latent = latent
        self.learning_rate = learning_rate
        self.weight = weight
        self.num_users, self.num_items = rating.shape
        self.epochs = epochs
        self.index_i = 0
        self.index_j = 0
        
        
    def fit(self):
        
        self.users_latent = np.random.normal(size=(self.num_users, self.latent))   #latent matrix 생성
        self.items_latent = np.random.normal(size=(self.num_items, self.latent))
        
        self.users_bias = np.zeros(self.num_users)                                #bias 0으로 생성
        self.items_bias = np.zeros(self.num_items)
        self.bias = np.mean(self.rating[np.where(self.rating != 0)])
        print("heel")
        self.training_process = []
        
        for epoch in range(self.epochs):                                    # 총 epochs만큼 반복해서 SGD 이행
          
            non_zero_value = self.rating.nonzero()
            
           
            
            rand_number = random.randrange(0,len(non_zero_value[0]))
            
            self.index_i = non_zero_value[0][rand_number]
            self.index_j = non_zero_value[1][rand_number]
            
            self.gradient_descent(self.index_i,self.index_j, self.rating[self.index_i,self.index_j])
            print("hi")
            
            cost = self.cost()
            
            self.training_process.append((epoch, cost))
            
            print("hello")
           
            
    def cost(self):
        
        
        prediction = self.calculated_matrix()
        cost = 0
        
        
        cost += pow(self.rating[self.index_i,self.index_j] - prediction[self.index_i,self.index_j],2)
            
        rmse_value = np.sqrt(cost)
        
        return rmse_value
    
    
    def calculated_matrix(self):
        
        return self.bias + self.users_bias[:,np.newaxis] + self.items_bias[np.newaxis, : ] + self.users_latent.dot(self.items_latent.T)
        
        
        
        
        
    def gradient(self, error, user_index, item_index):
        
        d_users = (error * self.items_latent[item_index,:]) - (self.weight * self.users_latent[user_index, :])
        d_items = (error * self.users_latent[user_index, :]) - (self.weight * self.items_latent[item_index, :])
        
        return d_users,d_items
    
    def gradient_descent(self,user_index, item_index, rating):
        
        prediction = self.predicted(user_index,item_index)
        error = rating - prediction
        
        self.users_bias[user_index] -= self.learning_rate * (error - self.weight*self.users_bias[user_index])
        self.items_bias[item_index] -= self.learning_rate * (error - self.weight*self.items_bias[item_index])
        
        d_users,d_items = self.gradient(error, user_index,item_index)
        
        self.users_latent[user_index, : ] -= self.learning_rate * d_users
        self.items_latent[item_index, : ] -= self.learning_rate * d_items
        
        
        
    def predicted(self,user_index,item_index):
        
        prediction_values = self.bias + self.users_bias[user_index]+self.items_bias[item_index] + self.users_latent[user_index,:].dot(self.items_latent[item_index, : ].T)
        
        return prediction_values
    

    
    def print
    

        
        

In [12]:
model = MatrixFactorization(ratings,latent=5, learning_rate = 0.5, weight = 0.5, epochs = 300,verbose = True)

model.fit() 

model.calculated_matrix()




heel
hi
hello


array([[ 5.72948513,  3.1907328 ,  1.99781219, ...,  5.36233428,
         4.78222365,  5.74290921],
       [ 3.17537237,  0.486741  , 10.80208567, ..., -0.52419668,
         1.00092406,  2.49500565],
       [ 4.78029935,  1.1367924 ,  1.11156723, ...,  9.06859797,
         4.958724  ,  5.21594105],
       ...,
       [ 1.84295605,  3.03493335,  6.63701205, ...,  1.99522584,
         2.25951308,  2.05434536],
       [ 5.27250584,  6.66913013,  3.55189213, ...,  0.94511519,
         4.16943589,  2.28651622],
       [ 2.78993067,  0.13325008,  7.16958077, ...,  2.7615894 ,
        -0.70965863,  3.02426453]])

In [ ]:
print(model.cost())

In [ ]:
model = MatrixFactorization(ratings,latent=5, learning_rate = 0.5, weight = 0.5, epochs = 300,verbose = True)

model.fit() 

model.calculated_matrix()
